# Global Bias Method
- Idea: Each user and each item has a specific bias towards ratings

## Packages

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import import_data_to_matrix, extract_submission
from utils import NUMBER_OF_MOVIES, NUMBER_OF_USERS

## Basic Settings
- $λ = 0.01$, 20 ALS (global bias) iterations 

In [ ]:
lambda_ = 0.01
iterations = 20

## Data Preprocessings
- Extract data to row-column format
- Impute missing data with 0

- Rating matrix A

In [ ]:
A = import_data_to_matrix()

- Observation matrix Ω

In [ ]:
W = (A > 0).astype(int)

## Global Bias
- Each user and each item has a specific bias towards ratings

- Global rating mean:
$$μ = \frac{\sum_{u=1}^n \sum_{i=1}^m ω_{ui}a_{ui}}{\sum_{u=1}^n \sum_{i=1}^m ω_{ui}}$$
- User $u$ rating mean:
$$μ_{(u,.)} = \frac{\sum_{i=1}^m ω_{ui}a_{ui}}{\sum_{i=1}^m ω_{ui}}$$
- Item $i$ rating mean:
$$μ_{(.,i)} = \frac{\sum_{u=1}^n ω_{ui}a_{ui}}{\sum_{u=1}^n ω_{ui}}$$
- User $u$ rating bias: (Initial values for ALS)
$$b_{(u,.)} = μ_{(u,.)} - \frac{\sum_{v=1}^n μ_{(v,.)}}{n}$$
- Item $i$ rating bias: (Initial values for ALS)
$$b_{(.,i)} = μ_{(.,i)} - \frac{\sum_{j=1}^m μ_{(.,j)}}{m}$$

In [ ]:
global_mean = np.sum(W * A)/np.sum(W)
Mu = np.array([np.sum(Wu * Au)/np.sum(Wu) for Au, Wu in zip(A, W)])
Mi = np.array([np.sum(Wi * Ai)/np.sum(Wi) for Ai, Wi in zip(A.T, W.T)])

Bu = Mu - np.mean(Mu)
Bi = Mi - np.mean(Mi)

Bu = np.reshape(Bu, (Bu.shape[0],1))
Bi = np.reshape(Bi, (Bi.shape[0],1))

## ALS
- Optimize global bias

- Objective function:
$$l(B_u, B_i) = \frac{1}{2}||Π_{Ω}(A - μ^{n⨉m} - B_u·1^{1⨉m} - (B_i·1^{1⨉n})^T)||_{F}^{2} + \frac{λ}{2}(||B_u||_{2}^{2} + ||B_i||_{2}^{2})$$

- $B_u$ is the column vector for all $b_{(u,.)}$ ($b_{(1,.)},...,b_{(n,.)}$)
- $B_i$ is the column vector for all $b_{(.,i)}$ ($b_{(.,1)},...,b_{(.,m)}$)


In [ ]:
def loss(A, Bu, Bi, W, l, global_mean):
    return ((1/2) * np.sum((W * (A - global_mean - np.dot(Bu, np.ones((1, NUMBER_OF_MOVIES))) - np.dot(Bi, np.ones((1, NUMBER_OF_USERS))).T) ** 2))
            + (l/2) * (np.sum(Bu ** 2) + np.sum(Bi ** 2)))

- Focus on the contribution to the error of a single $b_{(u,.)}$:
- Where does $b_{(u,.)}$ appear in error?
$$l_{B_i}(b_{(u,.)}) = \frac{1}{2}\sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(u,.)} - b_{(.,i)})^2 + \frac{λ}{2}b_{(u,.)}^2$$

- Partial derivative with respect to $b_{(u,.)}$
$$\frac{∂l}{∂b_{(u,.)}} = \frac{1}{2}\sum_{i=1}^m 2ω_{ui}(a_{ui} - μ - b_{(u,.)} - b_{(.,i)})(-1) + \frac{λ}{2}2b_{(u,.)}$$

- Set $\frac{∂l}{∂b_{(u,.)}} = 0$
$$0 = -\sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(u,.)} - b_{(.,i)}) + λb_{(u,.)}$$
$$λb_{(u,.)} = \sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)}) - \sum_{i=1}^m ω_{ui}b_{(u,.)}$$
$$λb_{(u,.)} + \sum_{i=1}^m ω_{ui}b_{(u,.)} = \sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)})$$
$$b_{(u,.)}(λ + \sum_{i=1}^m ω_{ui}) = \sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)})$$

- Thus,
$$b_{(u,.)}^* = (λ + \sum_{i=1}^m ω_{ui})^{-1}\sum_{i=1}^m ω_{ui}(a_{ui} - μ - b_{(.,i)})$$
- Note that $(λ + \sum_{i=1}^m ω_{ui}) > 0$ because $ω_{ui}∈\{0,1\}$ and $λ > 0$

- Equations for all $b_{(u,.)}$, $1<=u<=n$
$$(λ + \sum_{i=1}^m ω_{1i})b_{(1,.)} = \sum_{i=1}^m ω_{1i}(a_{1i} - μ - b_{(.,i)})$$
$$⋮$$
$$(λ + \sum_{i=1}^m ω_{ni})b_{(n,.)} = \sum_{i=1}^m ω_{ni}(a_{ni} - μ - b_{(.,i)})$$

- Thus,
$$\begin{bmatrix} (λ + \sum_{i=1}^m ω_{1i}) \\ ⋮ \\ (λ + \sum_{i=1}^m ω_{ni}) \end{bmatrix}\begin{bmatrix} b_{(1,.)}^* \\ ⋮ \\ b_{(n,.)}^* \end{bmatrix} = \begin{bmatrix} \sum_{i=1}^m ω_{1i}(a_{1i} - μ - b_{(.,i)}) \\ ⋮ \\ \sum_{i=1}^m ω_{ni}(a_{ni} - μ - b_{(.,i)}) \end{bmatrix}$$
- Which is equivalent to,
$$(λ^{n⨉n} + diag(Ω·1^{m⨉1}))\begin{bmatrix} b_{(1,.)}^* \\ ⋮ \\ b_{(n,.)}^* \end{bmatrix} = (Π_Ω(A - μ^{n⨉m} - 1^{n⨉1}·B_i^T))·1^{m⨉1}$$

- Similarly for all $b_{(.,i)}$,
$$(λ^{m⨉m} + diag(Ω^T·1^{n⨉1}))\begin{bmatrix} b_{.,1)}^* \\ ⋮ \\ b_{(.,m)}^* \end{bmatrix} = (Π_{Ω^T}(A^T - μ^{m⨉n} - 1^{m⨉1}·B_u^T))·1^{n⨉1}$$

In [ ]:
for epoch in tqdm(range(iterations)):
    Bi = np.linalg.solve(lambda_ + np.diag(np.dot(W.T, np.ones((NUMBER_OF_USERS,1))).T[0]),
                            np.dot(
                                W.T * (A.T - global_mean - np.dot(np.ones((NUMBER_OF_MOVIES, 1)), Bu.T)),
                                np.ones((NUMBER_OF_USERS, 1))
                            )
                        )

    print("Loss l(Bu,Bi) after solving for Bi:", loss(A, Bu, Bi, W, lambda_, global_mean))

    Bu = np.linalg.solve(lambda_ + np.diag(np.dot(W, np.ones((NUMBER_OF_MOVIES,1))).T[0]),
                            np.dot(
                                W * (A - global_mean - np.dot(np.ones((NUMBER_OF_USERS, 1)), Bi.T)),
                                np.ones((NUMBER_OF_MOVIES, 1))
                            )
                        )

    print("Loss l(Bu,Bi) after solving for Bu:", loss(A, Bu, Bi, W, lambda_, global_mean))

- Global estimation for rating of item $i$ by user $u$:
$$b_{ui} = μ + b_{(u,.)} + b_{(.,i)}$$
- Reconstruct data from the result of ALS (global bias) after 20 iterations (only not observed).

In [ ]:
biases = global_mean + np.array([Bu.T[0]]*NUMBER_OF_MOVIES).T + np.array([Bi.T[0]]*NUMBER_OF_USERS)

rec_A = A + (biases * (W^1))

## Export Predictions


In [ ]:
extract_submission(rec_A, file="global")